In [1]:

class SpiderCroppedVideoReader():
    """ This class mimics OpenCV's VideoCapture class, but instead reads
        the *.ufmf files, which contain 200x200 cropped/rotated and monochrome
        spider images.
    """

    def __init__(self, filename, progressCallback=None):
        self.fname = filename
        self.iframe = 0
        self.progressCallback = progressCallback

        # DEBUG
        self.MAX_NUM_FRAMES = 50 * 3600 * 24 * 100

        # Is this a supported input file?
        if not self.fname.endswith('.ufmf'):
            raise Exception('Unsupported input file. Only *.ufmf files allowed.')

        # Open file to determine size
        from motmot.SpiderMovie import SpiderMovie
        mov = SpiderMovie(filename)
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r')
        #N = self.arr.shape[0]
        N = mov.shape[0]*mov.shape[1]*mov.shape[2]
        #del self.arr
        # Ensure this file has the right size
        if N % (1024 * 1024) != 0:
            raise Exception('Image does not have expected size of 1024x1024.')
        # Open again with right shape
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r', shape=(int(N / (1024 * 1024)), 1024, 1024))
        self.matrix_shape = mov.shape
        self.mov = mov

    # Get various metadata
    def get(self, i):
        # Call progress callback
        if self.progressCallback is not None:
            try:
                self.progressCallback(float(i) / self.matrix_shape[0])
            except:
                pass
        # NUM. FRAMES
        if i == 7:
            if self.matrix_shape is not None:
                return min(self.MAX_NUM_FRAMES, self.matrix_shape[0])
            else:
                raise Exception('File closed.')
        # FPS
        elif i == 5:
            return 50
        # HEIGHT
        elif i == 4:
            return 1024
        # WIDTH
        elif i == 3:
            return 1024
        # ERROR
        else:
            raise Exception('Unsupported metadata requested: {}'.format(i))

    def isOpened(self):
        return True

    def read(self):
        if self.iframe >= self.get(7):
            return False, np.zeros((1024, 1024), dtype=np.uint8)
        else:
            self.iframe += 1
            return True, self.mov[self.iframe - 1]

In [2]:
basepath = 'Z:/HsinYi/Test Video From Darya/test_clip_dlc/'
project_name = 'TEST'
scorer = 'PC'

import os, glob


video_list = glob.glob(os.path.join(basepath, 'raw/')+'*.s.ufmf')
video_list


['Z:/HsinYi/Test Video From Darya/test_clip_dlc/raw\\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf']

In [3]:
import deeplabcut
deeplabcut.create_new_project(project_name , scorer, video_list, working_directory=basepath, copy_videos= True, videoReader = SpiderCroppedVideoReader)

Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\videos"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\labeled-data"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\training-datasets"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\dlc-models"
Copying the videos
\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\videos\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf
Generated "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\config.yaml"

A new project with name TEST-PC-2023-09-23 is created at \\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have chan

c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


'\\\\10.99.66.32\\Team Spider\\HsinYi\\Test Video From Darya\\test_clip_dlc\\TEST-PC-2023-09-23\\config.yaml'

In [4]:
config_path = basepath+ '/TEST-PC-2023-09-23/config.yaml'
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)
cfg['bodyparts'] =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']
deeplabcut.auxiliaryfunctions.write_config(config_path, cfg)

In [2]:
## Skip extract frames
## Skip label frames

## Generate labeling csv
import os, glob

video_list = glob.glob(os.path.join(basepath, 'raw/')+'*.s.ufmf')
videoname = video_list[0].split('raw\\')[1]
joint_filename = basepath + '/croprot/' +'bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s_dlc_abs.npy'

from deeplabcut.generate_training_dataset import creating_labeling_csv
creating_labeling_csv.creating_labeling_csv(config_path, basepath, videoname, joint_filename)


c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


CREATING-SOME LABELS FOR THE FRAMES


In [2]:
deeplabcut.check_labels(config_path, basepath=basepath)

Creating images with labels by PC.
\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\labeled-data\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf_labeled  already exists!


c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


They are stored in the following folder: \\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\labeled-data\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


In [1]:
import deeplabcut
basepath = 'Z:/HsinYi/Test Video From Darya/test_clip_dlc/'
config_path = basepath+ '/TEST-PC-2023-09-23/config.yaml'
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)

In [ ]:
print("CREATING TRAININGSET")
deeplabcut.create_training_dataset(config_path)

CREATING TRAININGSET
\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-23\training-datasets\iteration-0\UnaugmentedDataSet_TESTSep23  already exists!


c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value
C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\generate_training_dataset\trainingsetmanipulation.py:605: RuntimeWarning: invalid value encountered in less
  if Data[bodypart]['x'][jj]<np.shape(im)[1] and Data[bodypart]['y'][jj]<np.shape(im)[0]: #are labels in image?


In [ ]:
print('Train the network')
deeplabcut.train_network(config_path)